In [1]:
import pandas as pd
import json
from PIL import Image


In [2]:
csv_path = '/data/AlgProj/tct_yaoms/data/tct_0513/label.csv'
csv = pd.read_csv(csv_path)

In [4]:
import os
import pandas as pd
import json
from PIL import Image
import random

class TCTFilePreprocess():
    def __init__(self, data_dir, csv_path, save_dir):
        self.data_dir = data_dir
        self.labels = pd.read_csv(csv_path, engine='python')
        self.save_dir = save_dir
        self.series_id = self.labels['序列编号'].unique()
        self.drop_indexes = []
        
        self.labels['trusted'] = 0
        self.float2int()

    
    def float2int(self):
        self.labels['x_min'] = 0
        self.labels['y_min'] = 0
        self.labels['x_max'] = 0
        self.labels['y_max'] = 0
        self.labels['z'] = 0
        for idx in range(len(self.labels)):
            cur_dict = json.loads(self.labels.iloc[idx,:]['影像结果'])
            self.labels.loc[idx,'x_min'] = int(cur_dict['point1']['x'])
            self.labels.loc[idx,'y_min'] = int(cur_dict['point1']['y'])
            self.labels.loc[idx,'x_max'] = int(cur_dict['point2']['x'])
            self.labels.loc[idx,'y_max'] = int(cur_dict['point2']['y'])
            self.labels.loc[idx, 'z'] = int(cur_dict['point1']['z'])

    def remove_untrusted_label(self):
        for idx in self.labels.index:
            if self.labels.loc[idx, '恶性细胞分类'] == 10:
                self.labels = self.labels.drop(index=idx)
        #reset index
        self.labels.reset_index(drop=True)
        

    def produce_new_label(self):
        columns = ['序列编号','影像结果编号','x_min','y_min','x_max','y_max','z','恶性细胞分类']
        path = self.save_dir + 'detail_infomation.csv'
        if not self.check_exist(path):
#             drop_csv = self.labels.drop(index=self.drop_indexes,axis=0)
#             print(len(drop_csv))
            print('detail_information.csv 生成中')
            self.labels.to_csv(path,columns=columns, index=False)
        else:
            print('detail_infomation.csv文件已存在')
        
    
    def crop_malignant(self):
        #crop img
        df = pd.DataFrame(columns=['img_path', 'label'])
        
        for idx in self.labels.index:
            series = self.labels.loc[idx, '序列编号']
            results_idx = self.labels.loc[idx, '影像结果编号']
            label = self.labels.loc[idx, '恶性细胞分类'].item()
            z = str(self.labels.loc[idx, 'z'])
            len_z = len(z)
            img_name = '0'*(6 - len_z) + z +'.jpg'
            img_path = self.data_dir + series + '/' + img_name
            
            save_path = self.save_dir+ 'tct_0513_crop_imgs/' + str(results_idx) +'_'+series + '_'+ img_name
            df = df.append([{'img_path':save_path, 'label':label}], ignore_index=True)
            
            if not self.check_exist(save_path):
                img = Image.open(img_path)
                shape = img.size

                x_min = max(0, self.labels.loc[idx, 'x_min'])
                y_min = max(0, self.labels.loc[idx, 'y_min'])
                x_max = min(shape[0]-1, self.labels.loc[idx, 'x_max'])
                y_max = min(shape[1]-1, self.labels.loc[idx, 'y_max'])
                box = [x_min, y_min,x_max, y_max]
                print(shape, box)

                crop_img = img.crop(box)                        
                crop_img.save(save_path, quality=95)
            else:
                continue
            
            df.to_csv(self.save_dir+'dataset.csv', index=False)
                        
    
    def check_exist(self, path):
        return os.path.exists(path)

In [5]:
data_dir = '/data/AlgProj/niezl/TCT/1_DataBase/20190711/'
save_dir = '/data/AlgProj/tct_yaoms/data/tct_0513/'
csv_path = '/data/AlgProj/tct_yaoms/data/tct_0513/label.csv'

In [6]:
tct_0513 = TCTFilePreprocess(data_dir,  csv_path, save_dir)

In [17]:
tct_0513.labels.tail()

,序列编号,影像结果,恶性细胞分类,影像结果编号,trusted,x_min,y_min,x_max,y_max,z
2374,06d40c98aa124d278eb9422b2bc0d3d8,"{""point1"":{""x"":414.5103448275862,""y"":46.124137...",1,332260,0,414,46,601,327,420
2375,06d40c98aa124d278eb9422b2bc0d3d8,"{""point1"":{""x"":137.7655172413793,""y"":324.96551...",1,332257,0,137,324,378,574,432
2376,06d40c98aa124d278eb9422b2bc0d3d8,"{""point1"":{""x"":5.682758620689668,""y"":272.55172...",1,332256,0,5,272,274,553,376
2377,06d40c98aa124d278eb9422b2bc0d3d8,"{""point1"":{""x"":43.42068965517245,""y"":475.91724...",1,332259,0,43,475,198,696,414
2378,06d40c98aa124d278eb9422b2bc0d3d8,"{""point1"":{""x"":695.4482758620691,""y"":197.07586...",1,332258,0,695,197,923,371,465


In [7]:
tct_0513.remove_untrusted_label()

In [9]:
tct_0513.produce_new_label()

detail_information.csv 生成中


In [15]:
tct_0513.crop_malignant()

(1936, 1216) [1800, 8, 1934, 171]
(1936, 1216) [670, 602, 1000, 1096]
(1936, 1216) [168, 572, 385, 818]
(1936, 1216) [1274, 634, 1907, 936]
(1936, 1216) [426, 578, 571, 660]
(1936, 1216) [77, 662, 250, 854]
(1936, 1216) [870, 496, 1254, 759]
(1936, 1216) [43, 1068, 398, 1210]
(1936, 1216) [978, 39, 1136, 212]
(1936, 1216) [297, 753, 584, 910]
(1936, 1216) [1649, 537, 1897, 768]
(1936, 1216) [183, 367, 767, 1046]
(1936, 1216) [1149, 233, 1392, 563]
(1936, 1216) [178, 313, 717, 645]
(1936, 1216) [344, 512, 513, 703]
(1936, 1216) [526, 522, 1065, 822]
(1936, 1216) [1220, 233, 1338, 382]
(1936, 1216) [32, 609, 480, 977]
(1936, 1216) [125, 440, 265, 602]
(1936, 1216) [487, 956, 631, 1104]
(1936, 1216) [644, 1044, 758, 1117]
(1936, 1216) [1560, 719, 1759, 1033]
(1936, 1216) [1606, 380, 1813, 587]
(1936, 1216) [327, 563, 795, 796]
(1936, 1216) [532, 205, 756, 542]
(1936, 1216) [610, 878, 860, 1089]
(1936, 1216) [73, 423, 469, 716]
(1936, 1216) [301, 91, 767, 555]
(1936, 1216) [17, 561, 299, 8

(1936, 1216) [834, 112, 1055, 333]
(1936, 1216) [1279, 96, 1449, 207]
(1936, 1216) [1732, 259, 1898, 403]
(1936, 1216) [564, 792, 662, 908]
(1936, 1216) [289, 872, 367, 986]
(1936, 1216) [1554, 163, 1692, 273]
(1936, 1216) [1121, 1076, 1197, 1193]
(1936, 1216) [1690, 427, 1796, 549]
(1936, 1216) [812, 417, 956, 603]
(1936, 1216) [9, 319, 77, 519]
(1936, 1216) [431, 834, 560, 934]
(1936, 1216) [1500, 118, 1724, 303]
(1936, 1216) [566, 616, 648, 692]
(1936, 1216) [780, 68, 1115, 313]
(1936, 1216) [1411, 112, 1810, 487]
(1936, 1216) [1337, 403, 1754, 682]
(1936, 1216) [694, 573, 856, 698]
(1936, 1216) [287, 842, 399, 978]
(1936, 1216) [1427, 321, 1586, 419]
(1936, 1216) [860, 163, 960, 289]
(1936, 1216) [1279, 1092, 1391, 1153]
(1936, 1216) [1502, 479, 1794, 870]
(1936, 1216) [934, 708, 1111, 932]
(1936, 1216) [1828, 22, 1926, 309]
(1936, 1216) [1452, 303, 1580, 419]
(1936, 1216) [1267, 912, 1433, 1097]
(1936, 1216) [1397, 632, 1540, 770]
(1936, 1216) [197, 124, 325, 239]
(1936, 1216) [14

(1936, 1216) [120, 486, 224, 643]
(1936, 1216) [1632, 852, 1772, 979]
(1936, 1216) [325, 906, 429, 1012]
(1936, 1216) [21, 548, 107, 684]
(1936, 1216) [21, 790, 79, 895]
(1936, 1216) [181, 559, 312, 701]
(1936, 1216) [196, 61, 291, 143]
(1936, 1216) [400, 164, 489, 274]
(1936, 1216) [194, 688, 351, 805]
(1936, 1216) [280, 348, 452, 561]
(1936, 1216) [476, 464, 627, 611]
(1936, 1216) [77, 682, 215, 824]
(1936, 1216) [586, 52, 668, 195]
(1936, 1216) [993, 576, 1127, 736]
(1936, 1216) [254, 445, 390, 591]
(1936, 1216) [560, 361, 733, 527]
(1936, 1216) [1586, 846, 1752, 1001]
(1936, 1216) [341, 605, 439, 746]
(1936, 1216) [109, 811, 222, 919]
(1936, 1216) [1494, 141, 1675, 268]
(1936, 1216) [485, 749, 627, 941]
(1936, 1216) [400, 337, 562, 471]
(1936, 1216) [1783, 368, 1915, 518]
(1936, 1216) [31, 754, 194, 996]
(1936, 1216) [1723, 217, 1897, 322]
(1936, 1216) [1376, 918, 1454, 1064]
(1936, 1216) [1625, 28, 1830, 283]
(1936, 1216) [1744, 993, 1844, 1098]
(1936, 1216) [216, 110, 315, 176]
(

(1936, 1216) [1283, 30, 1408, 126]
(1936, 1216) [1053, 326, 1214, 466]
(1936, 1216) [266, 120, 559, 399]
(1936, 1216) [369, 31, 749, 433]
(1936, 1216) [71, 479, 323, 789]
(1936, 1216) [458, 465, 609, 617]
(1936, 1216) [456, 642, 707, 825]
(1936, 1216) [1054, 850, 1388, 1050]
(1936, 1216) [683, 786, 925, 1042]
(1936, 1216) [811, 8, 1035, 129]
(1936, 1216) [575, 417, 920, 656]
(1936, 1216) [1234, 679, 1521, 937]
(1936, 1216) [1454, 234, 1662, 465]
(1936, 1216) [334, 669, 563, 875]
(1936, 1216) [176, 15, 447, 205]
(1936, 1216) [1672, 743, 1869, 935]
(1936, 1216) [971, 521, 1221, 795]
(1936, 1216) [1060, 837, 1248, 1093]
(1936, 1216) [55, 518, 249, 742]
(1936, 1216) [841, 346, 975, 594]
(1936, 1216) [21, 789, 211, 978]
(1936, 1216) [927, 275, 1271, 569]
(1936, 1216) [742, 161, 1212, 417]
(1936, 1216) [845, 896, 1067, 1102]
(1936, 1216) [275, 695, 490, 940]
(1936, 1216) [994, 227, 1299, 589]
(1936, 1216) [69, 271, 550, 585]
(1936, 1216) [360, 203, 652, 468]
(1936, 1216) [4, 704, 392, 1042]


(1936, 1216) [1716, 316, 1856, 433]
(1936, 1216) [655, 651, 795, 804]
(1936, 1216) [1744, 336, 1897, 515]
(1936, 1216) [676, 748, 821, 944]
(1936, 1216) [899, 2, 1162, 215]
(1936, 1216) [711, 401, 858, 573]
(1936, 1216) [325, 646, 420, 748]
(1936, 1216) [515, 838, 670, 970]
(1936, 1216) [174, 713, 310, 847]
(1936, 1216) [1097, 181, 1224, 375]
(1936, 1216) [814, 750, 998, 1093]
(1936, 1216) [1403, 687, 1526, 819]
(1936, 1216) [1698, 338, 1914, 495]
(1936, 1216) [618, 629, 685, 795]
(1936, 1216) [711, 582, 847, 771]
(1936, 1216) [1323, 338, 1591, 748]
(1936, 1216) [325, 784, 444, 974]
(1936, 1216) [1675, 183, 1752, 349]
(1936, 1216) [864, 377, 961, 547]
(1936, 1216) [1457, 301, 1571, 461]
(1936, 1216) [1407, 498, 1524, 577]
(1936, 1216) [560, 631, 704, 754]
(1936, 1216) [1274, 288, 1474, 541]
(1936, 1216) [562, 636, 830, 968]
(1936, 1216) [532, 907, 704, 1011]
(1936, 1216) [1246, 174, 1360, 368]
(1936, 1216) [1789, 40, 1886, 235]
(1936, 1216) [390, 181, 495, 299]
(1936, 1216) [797, 1062,

In [8]:
len(tct_0513.labels)

2305

In [17]:
len(os.listdir( '/data/AlgProj/tct_yaoms/data/tct_0513/tct_0513_crop_imgs'))

2305

In [19]:
pd.read_csv('/data/AlgProj/tct_yaoms/data/tct_0513/dataset.csv').tail()

,img_path,label
2300,/data/AlgProj/tct_yaoms/data/tct_0513/tct_0513...,1
2301,/data/AlgProj/tct_yaoms/data/tct_0513/tct_0513...,1
2302,/data/AlgProj/tct_yaoms/data/tct_0513/tct_0513...,1
2303,/data/AlgProj/tct_yaoms/data/tct_0513/tct_0513...,1
2304,/data/AlgProj/tct_yaoms/data/tct_0513/tct_0513...,1
